<a href="https://colab.research.google.com/github/marzinouri/AzeriPipeline/blob/main/Notebooks/MachineTranslation/Translation_fa2azb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Acknowledgment: This portion of the code is based on the work available at [JoeyNMT](https://github.com/joeynmt/joeynmt).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/translation

# Prerequisites

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/translation"

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
import torch
torch.__version__

'1.13.1+cu116'

# Data Preparation

In [ ]:
import os
from pathlib import Path
import pandas as pd
import random


def load_data_to_df(path):
    """
    Load dataset
    """
    lines = Path(path).open(encoding="utf-8").read().strip().split("\n")

    data = {
        "id": range(len(lines)),
    }

    #load data into a DataFrame object:
    df = pd.DataFrame(data)
    sents = []
    for i, l in enumerate(lines):
        lang1, lang2 = l.split("\t")
        sents.append({"azb": lang1, "fa": lang2})
    random.seed(42)
    random.shuffle(sents)
    df["translation"] = sents

    return df

In [ ]:
df = load_data_to_df("/content/drive/MyDrive/Preprocessed_Datasets/Bilingual/ALL_v2.tsv")

In [ ]:
from datasets import Dataset

train_frac = 0.8
dev_frac = 0.1
test_frac = 0.1

total = len(df)
train_num = int(train_frac * len(df))
dev_num = int(dev_frac * len(df))

data_train = Dataset.from_pandas(df[:train_num+1])
data_dev = Dataset.from_pandas(df[train_num+1:train_num+1+dev_num])
data_test = Dataset.from_pandas(df[train_num+1+dev_num:])

data_train, data_dev, data_test

(Dataset({
     features: ['id', 'translation'],
     num_rows: 11978
 }), Dataset({
     features: ['id', 'translation'],
     num_rows: 1497
 }), Dataset({
     features: ['id', 'translation'],
     num_rows: 1497
 }))

Inspect the data



In [ ]:
data_train['translation'][:3]

[{'azb': 'اونا یئشآیا پیغمبرین کیتابینی وئردیلر . ایسا تومارێ آچێب بو سؤزلر یازیلان هیسسهنی تاپدێ : ',
  'fa': 'طومار اشعْیای نبی به او داده شد ، او طومار را گشود و بخشی را یافت که چنین نوشته شده است : '},
 {'azb': 'وه اؤزو ده جهنهمه آتیلاجاقدیر . ',
  'fa': 'و فرجامش\u200c درافتادن به جهنم است . '},
 {'azb': 'البته عيلتی ده بودور کی', 'fa': 'البته علتش هم اين است كه '}]

In [ ]:
data_dev['translation'][:3]

[{'azb': 'بئلهجه یهودا بیر آلای اسگرله باشچێ کاهنلرین وه فاریسئیلرین گؤندردیگی بزی مۆحافظهچیلری گؤتوروب چێراقلار ، مشلر وه سلاحلارلا اورایا گلدی . ',
  'fa': 'پس یهودا گروهی از سربازان و مأموران سران کاهنان و فریسیان را با خود همراه کرد و آنان با مشعل و چراغ و سلاح به آنجا رسیدند . '},
 {'azb': 'دئدیم : ائله بیل یاتمیشدین آییلدین هه ! ',
  'fa': 'گفتم\u200c : خوابت\u200c پرید ! '},
 {'azb': 'بیز کیتابدان سونرا زبوردا دا تورپاغا یالنیز منیم سالئه بندهلریمین داخل اولاجاغێنێ یازمیشدیق . ',
  'fa': 'و در حقیقت ، در زبور پس از تورات نوشتیم که زمین را بندگان شایسته ما به ارث خواهند برد . '}]

In [ ]:
data_test['translation'][:3]

[{'azb': 'ائشیتدیک کی ، بیزلردن بزی آداملار یانینیزا گلیب سؤزلری ایله سیزی لرزهیه سالاراق دۆشۆنجهلرینیزی قارێشدێرێبلار . لاکین بونو اونلارا بیز تاپشێرمامێشدێق . ',
  'fa': 'شنیده\u200cایم که بعضی از میان ما ، هرچند که ما به آنان حکم نکرده بودیم ، شما را با سخنان خود مضطرب ساخته\u200cاند و در تلاش بوده\u200cاند ذهنتان را آشفته سازند . '},
 {'azb': 'آمما اؤز دینینی آرالارێندا پارچالاییب فرقه فرقه اولدولار . هر فرقه اؤز دینینه سئوینیر ',
  'fa': 'تا کار دین\u200c شان را میان خود قطعه قطعه کردند و دسته دسته شدند : هر دسته\u200cای به آنچه نزدشان بود ، دل خوش کردند . '},
 {'azb': 'گۆندهلیک چؤرهییمیزی بیزه بو گۆن وئر . ',
  'fa': 'نان روزانهٔ ما را امروز به ما عطا کن . '}]

Save the train-dev-test splits in local dir

In [ ]:
from datasets.dataset_dict import DatasetDict

dataset_dict = DatasetDict({
  "train": data_train,
  "validation": data_dev,
  "test": data_test
})

data_dir = "RESULTS_fa2azb/data"
dataset_dict.save_to_disk(data_dir)

Saving the dataset (0/1 shards):   0%|          | 0/11978 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1497 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1497 [00:00<?, ? examples/s]

## Build Vocab

In [ ]:
src = 'fa'
trg = 'azb'

In [ ]:
from pathlib import Path

# Create the config
config = f"""
name: "data_sp"
joeynmt_version: "2.0.0"

data:
    train: "{data_dir}/train"
    dev: "{data_dir}/validation"
    test: "{data_dir}/test"
    dataset_type: "huggingface"
    sample_dev_subset: 200
    src:
        lang: "{src}"
        max_length: 100
        lowercase: False
        normalize: False
        level: "bpe"
        voc_limit: 2000
        voc_min_freq: 1
        voc_file: "{data_dir}/vocab.txt"
        tokenizer_type: "sentencepiece"
        tokenizer_cfg:
            model_file: "{data_dir}/sp.model"

    trg:
        lang: "{trg}"
        max_length: 100
        lowercase: False
        normalize: False
        level: "bpe"
        voc_limit: 2000
        voc_min_freq: 1
        voc_file: "{data_dir}/vocab.txt"
        tokenizer_type: "sentencepiece"
        tokenizer_cfg:
            model_file: "{data_dir}/sp.model"

""".format(data_dir=data_dir)
with (Path(data_dir) / "config.yaml").open('w') as f:
    f.write(config)

In [ ]:
!python3 scripts/build_vocab.py {data_dir}/config.yaml --joint

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(
Dropping NaN...: 100% 12/12 [00:00<00:00, 100.90ba/s]
Preprocessing...: 100% 11978/11978 [00:01<00:00, 10124.44ex/s]
### Training sentencepiece...
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/tmp/sentencepiece_9ur21_0_.txt --model_prefix=RESULTS_fa2azb/data/sp --model_type=unigram --vocab_size=2000 --character_coverage=1.0 --accept_language=fa,azb --unk_piece=<unk> --bos_piece=<s> --eos_piece=</s> --pad_piece=<pad> --unk_id=0 --bos_id=2 --eos_id=3 --pad_id=1 --vocabulary_output_piece_score=false
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /tmp/sentencepiece_9ur21_0_.txt
  input_format: 
  model_prefix: RESULTS_fa2azb/data/sp
  model_type: U

In [ ]:
!head -10 {data_dir}/vocab.txt

<unk>
<pad>
<s>
</s>
▁،
ی
▁.
ه
ا
▁و


## Configuration


In [ ]:
model_dir = "RESULTS_fa2azb/model"
config += """
testing:
    n_best: 1
    beam_size: 5
    beam_alpha: 1.0
    batch_size: 512
    batch_type: "token"
    max_output_length: 100
    eval_metrics: ["bleu"]
    #return_prob: "hyp"
    #return_attention: False
    sacrebleu_cfg:
        tokenize: "13a"

training:
    #load_model: "{model_dir}/latest.ckpt"
    #reset_best_ckpt: False
    #reset_scheduler: False
    #reset_optimizer: False
    #reset_iter_state: False
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999]
    scheduling: "warmupinversesquareroot"
    learning_rate_warmup: 2000
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    loss: "crossentropy"
    batch_size: 512
    batch_type: "token"
    batch_multiplier: 4
    early_stopping_metric: "loss"
    epochs: 500
    updates: 2000000000
    validation_freq: 1000
    logging_freq: 100
    model_dir: "{model_dir}"
    overwrite: True
    shuffle: True
    use_cuda: True
    print_valid_sents: [0, 1, 2, 3]
    keep_best_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 2
        num_heads: 4
        embeddings:
            embedding_dim: 256
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256
        ff_size: 1024
        dropout: 0.1
        layer_norm: "pre"
    decoder:
        type: "transformer"
        num_layers: 2
        num_heads: 8
        embeddings:
            embedding_dim: 256
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256
        ff_size: 1024
        dropout: 0.1
        layer_norm: "pre"

""".format(model_dir=model_dir)
with (Path(data_dir) / "config.yaml").open('w') as f:
    f.write(config)

# Model

In [ ]:
!python3 -m joeynmt train {data_dir}/config.yaml

Streaming output truncated to the last 5000 lines.
2023-01-19 23:21:56,815 - INFO - joeynmt.prediction - Predicting 200 example(s)... (Greedy decoding with min_output_length=1, max_output_length=100, return_prob='none', generate_unk=True, repetition_penalty=-1, no_repeat_ngram_size=-1)
2023-01-19 23:22:01,214 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2023-01-19 23:22:01,214 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2023-01-19 23:22:01,214 - WARNING - sacrebleu - If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
2023-01-19 23:22:01,215 - INFO - joeynmt.metrics - nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1
2023-01-19 23:22:01,218 - INFO - joeynmt.prediction - Evaluation result (greedy) bleu:   7.53, loss:   2.75, ppl:  15.58, acc:   0.42, nltk_bleu:   0.00, nltk_gleu:   0.00, nltk_nist:   0.00, generation: 4.3584

## Continue training after interruption


In [ ]:
resume_config = config\
  .replace('#load_model:', 'load_model:')\
  .replace('#reset_best_ckpt: False', 'reset_best_ckpt: False')\
  .replace('#reset_scheduler: False', 'reset_scheduler: False')\
  .replace('#reset_optimizer: False', 'reset_optimizer: False')\
  .replace('#reset_iter_state: False', 'reset_iter_state: False')\
  .replace(f'model_dir: "{model_dir}"', f'model_dir: "{model_dir}_resume"')

with (Path(data_dir) / "resume_config.yaml").open('w') as f:
    f.write(resume_config)

In [ ]:
!python3 -m joeynmt train {data_dir}/resume_config.yaml

# Evaluation


In [ ]:
!python3 -m joeynmt test {data_dir}/config.yaml --ckpt {model_dir}/best.ckpt

2023-01-20 02:17:04,348 - INFO - root - Hello! This is Joey-NMT (version 2.0.0).
2023-01-20 02:17:04,349 - INFO - joeynmt.data - Building tokenizer...
2023-01-20 02:17:04,360 - INFO - joeynmt.tokenizers - fa tokenizer: SentencePieceTokenizer(level=bpe, lowercase=False, normalize=False, filter_by_length=(-1, 100), pretokenizer=none, tokenizer=SentencePieceProcessor, nbest_size=5, alpha=0.0)
2023-01-20 02:17:04,360 - INFO - joeynmt.tokenizers - azb tokenizer: SentencePieceTokenizer(level=bpe, lowercase=False, normalize=False, filter_by_length=(-1, 100), pretokenizer=none, tokenizer=SentencePieceProcessor, nbest_size=5, alpha=0.0)
2023-01-20 02:17:04,360 - INFO - joeynmt.data - Building vocabulary...
2023-01-20 02:17:04,405 - INFO - joeynmt.data - Loading dev set...
2023-01-20 02:17:04,538 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in ve

In [ ]:
!python3 -m joeynmt translate {data_dir}/config.yaml --ckpt {model_dir}/best.ckpt